# Задание 1

Аудио длинной в 1 секунду и частотой дискретизации в 16 кГЦ загоняем в SIFT с окном в 25 мс и шагом в 10 мс.\
Требуется найти количество поместившихся в него временных окон

In [1]:
def count_stft_frames(sample_rate, sample_duration, window_size, step_size):
    # переводим из секунд в количество семплов
    # длинна в секундах на частоту дискретизации в герцах(семплов в секунду)
    window_size = window_size * sample_rate
    step_size = step_size * sample_rate

    samples = sample_duration * sample_rate  # кличество семплов в аудио

    # считаем количество окон (сколько целых шагов нужно чтобы дойти до последнего + 1 последний)
    frames = (samples - window_size) // step_size + 1  # -> получем только окна которые полностью помещаются в сигнал

    return int(frames)  # frames уже целое из за целочисленного деления

In [2]:
sample_rate = 16000  # частота дискретизации в герцах
sample_duration = 1  # длительность сигнала в секундах
window_size = 0.025  # длина окна в секундах
step_size = 0.010  # шаг окна в секундах

In [3]:
frames = count_stft_frames(sample_rate, sample_duration, window_size, step_size)

print(f"Временных окон: {frames}")

Временных окон: 98


# Задание 2

Модель: y_pred = wx + b\
Функция потерь: L = (y_pred - y_true)^2

вес w = 2.0 и смещение b = 1.0

на входе x = 3\
ожидаемый предикт y_true = 10

Требуется посчитать градиенты для w и b

In [4]:
# модель y_pred = wx + b
class Model:
    def __init__(self, w, b):
        self.w = w
        self.b = b

    def forward(self, x):  # вычисление линейного выражения (предикта)
        y = self.w * x + self.b
        return y

    def backward(self, x, y_true):
        y_pred = self.forward(x)

        # производная по x от (wx+b - y_true)^2 = 2*(wx+b - y_true) * (производная по w от (wx+b - y_true) которая равна x)
        dL_dw = 2 * (y_pred - y_true) * x

        # производная по x от (wx+b - y_true)^2 = 2*(wx+b - y_true) * (производная по b от (wx+b - y_true) которая равна 1)
        dL_db = 2 * (y_pred - y_true)

        return dL_dw, dL_db

In [5]:
# лосс L = (y_pred - y_true)^2
def loss(model, x, y):
    L = (model.forward(x) - y) ** 2

    return L

In [6]:
x = 3
y_true = 10

In [7]:
model = Model(2.0, 1.0)

In [8]:
# вычислим лосс для предикта модели
L = loss(model, x, y_true)
print(L)

9.0


In [9]:
# найдем частные производные функции потерь по обучаемым параметрам
# dL по dw и dL по db
dL_dw, dL_db = model.backward(x, y_true)
print("dL по dw:", dL_dw)
print("dL по db:", dL_db)

dL по dw: -18.0
dL по db: -6.0


# Задание 3

Собираемся обучить модель для решения задачи распознавания речи.



In [ ]:
"""
a) Начнем с загрузки аудио и парсинга расшифровки. Приведем аудио к формату с которым может работать модель (разделим на термы нужной длинны)
   Можно нормализовать громкость звука и добавить шумоподавление чтобы модели было легче рязделять и понимать слова

b) Будем проходить по монологам и разбивать термы так чтобы их длинна не превышала 30 секунд
   Т.е монолог одного говорящего поделим на термы по 30 секунд разбивая аудио по словам с помощью librosa.effects.split
   Каждый терм дополним тишиной до 30 секунд
   Для каждого получившегося терма подберем слова из исходного (Каждая 30 секундная часть будет содержать слова которые в нее входят)

c) Можно использовать добавление шума разных частот к аудио, уменьшение / увеличение громкости, добавление звуковых эффектов например реверберации

d) Разделить датасет на трейн и валидацию для отслеживания реальных метрик
   Использовать аугментации для увеличения количества семплов в датасете
   Использовать регуляризации внутри модели. Попробовать дообучить предобученные модели

e) Для Whisper сохраним описания в формате текста и временных меток для аудио файла
"""

In [ ]:
# посмотрим на формат расшифровки:
"""
{
    "schemaVersion": "2.0",
    "monologues": [
        {
            "speaker": {
                "id": "0",
                "name": ""
            },
            "start": 0.0,
            "end": 33.221,
            "terms": [
                {
                    "text": "Еще большую тревогу вызывает у меня американская политика в отношении Ближнего Востока, которая исторически, а особенно после 11 сентября 2011 года, была излишне сфокусирована на подавлении любых политических волнений во имя препятствования «исламскому фундаментализму» – выражение, которое использовал практически каждый режим. Беда в том, что уничтожение исламистов увеличивает их численность; в любом случае Запад и его автократические арабские союзники лишь усилили исламских фундаменталистов, когда загнали их в подполье.",
                    "start": 0.0,
                    "end": 33.221,
                    "text_normalized": "еще большую тревогу вызывает у меня американская политика в отношении ближнего востока которая исторически а особенно после одиннадцатого сентября две тысячи одиннадцатого года была излишне сфокусирована на подавлении любых политических волнений во имя препятствования исламскому фундаментализму выражение которое использовал практически каждый режим беда в том что уничтожение исламистов увеличивает их численность в любом случае запад и его автократические арабские союзники лишь усилили исламских фундаменталистов когда загнали их в подполье "
                }
            ]
        },
        ...
"""
# Будем разделять по монологам. Монологи по термам.
# Каждый терм состоит из полей text_normalized представляющего собой речь нормализованную так как она произносится по словам. Т.е вырезаны все знаки припенания и числа находятся в форме слов.
# Также есть границы терма

In [ ]:
import librosa
import numpy as np
import soundfile as sf

import json
import os

In [ ]:
# Параметры
AUDIOT_FILE = "diff_spks_lvl_1_0_0_0_05.ogg"
TRANSCRIPT_FILE = "diff_spks_lvl_1_0_0_0_05.json"
MONOLOGUES_DIR = "monologues"
TERM_DURATION = 30.0
SAMPLE_RATE = 16000

In [ ]:
# загружаем аудио и вычисляем частоту дискретизации
audio, SAMPLE_RATE = librosa.load(AUDIOT_FILE, sr=SAMPLE_RATE)  # меняет частоту дискретизации на SAMPLE_RATE

# загружаем расшифровку
with open(TRANSCRIPT_FILE, 'r', encoding='utf-8') as f:
    transcript = json.load(f)

# создаем папку для монологов с термами длинной в TERM_DURATION и приложенным к ним расшифровкам
os.makedirs(MONOLOGUES_DIR, exist_ok=True)

In [ ]:
# аугментация по громкости
def aug_volume(audio, mindb=5, maxdb=5):
    # случайно измененяем громкось в диапозоне mindb : maxdb
    gain_db = np.random.uniform(mindb, maxdb)
    audio = librosa.effects.preemphasis(audio, coef=gain_db)

    return audio

# аугментация по шуму
def aug_noise(audio, noise_level=0.005):
    # добавляем белый шум
    noise = np.random.normal(0, noise_level, audio.shape)
    audio = audio + noise

    return audio

# применяем набор аугментаций к аудио
def aug(audio):
    audio_augms = [audio]  # вместе с оригиналом

    for aug in [aug_volume, aug_noise]:
        # Изменение громкости
        audio = aug(audio)
        audio_augms.append(audio)

    return audio_augms


In [ ]:
for monolog_index, monolog in enumerate(transcript["monologues"]):
    # создаем отдельные папки для каждого монолога
    monolog_path = f"{MONOLOGUES_DIR}/monologue_{monolog_index}"
    os.makedirs(monolog_path, exist_ok=True)

    # создадим описание монолога и будем заносить в него новые обработанные термы
    monolog_discr = {"speaker": monolog["speaker"], "start": monolog["start"], "end": monolog["end"], "terms": []}

    term_index = 0
    for term in monolog["terms"]:
        # Перевод время в количество сэмплов
        start_sample = int(term["start"] * SAMPLE_RATE)
        end_sample = int(term["end"] * SAMPLE_RATE)
        required_samples = int(TERM_DURATION * SAMPLE_RATE)

        # обрежем аудио
        term_audio = audio[start_sample:end_sample]

        # если терм вписывается в TERM_DURATION будем сохранять его как есть
        if term["end"] - term["start"] <= TERM_DURATION:
            # дополним его тишиной до TERM_DURATION
            if len(term_audio) < required_samples:
                padding = np.zeros(required_samples - len(term_audio))
                term_audio = np.concatenate([term_audio, padding])

            # и сохраним аугментации для терма
            audio_paths = []
            for aug_index, augm_audio in enumerate(aug(term_audio)):
                audio_path = f"{monolog_path}/term_{term_index}_aug{aug_index}.wav"
                audio_paths.append(audio_path)
                sf.write(audio_path, augm_audio, SAMPLE_RATE)

            # добавляем описание терма для Whisper
            monolog_discr["terms"].append({
                "audio_path": audio_paths[0],  # берем первый аудиофайл (или добавляем все для аугментаций)
                "start": term["start"],
                "end": term["end"],
                "text": term["text_normalized"]  # Используем нормализованный текст для обучения
            })

        # Если терм не вписывается в TERM_DURATION, будем делить его на несколько частей
        else:
            terms = []
            term_words = term["text_normalized"].split()
            cur_term_words = []
            terms_start = term["start"]
            terms_end = term["start"]

            segments = librosa.effects.split(term_audio, top_db=20)  # разделяем на сегменты по уровням громкости
            for segment_index, segment in enumerate(segments):  # проходим по сегментам-словам
                # Вычисляем начало и конец сегмента
                segment_start_sample = segment[0]
                segment_end_sample = segment[1]
                segment_start_time = (segment_start_sample / SAMPLE_RATE) + term["start"]
                segment_end_time = (segment_end_sample / SAMPLE_RATE) + term["start"]

                # Добавляем слово текущему терму, если длина терма со словом меньше TERM_DURATION
                if terms_end - terms_start + (segment_end_time - segment_start_time) <= TERM_DURATION:
                    terms.append(segment)
                    term_words.append(term_words[0])
                    term_words = term_words[1:]
                    terms_end = segment_end_time

                # Если слово не влезет в терм, то запишем текущий и начнем новый
                if terms_end - terms_start + (segment_end_time - segment_start_time) > TERM_DURATION or segment_index == len(segments) - 1:
                    start_sample = int(terms_start * SAMPLE_RATE)
                    end_sample = int(terms_end * SAMPLE_RATE)
                    required_samples = int(TERM_DURATION * SAMPLE_RATE)

                    chunk_audio = audio[start_sample:end_sample]

                    # дополним его тишиной до TERM_DURATION
                    if len(chunk_audio) < required_samples:
                        padding = np.zeros(required_samples - len(chunk_audio))
                        chunk_audio = np.concatenate([chunk_audio, padding])

                    # и сохраним аугментации для терма
                    audio_paths = []
                    for aug_index, augm_audio in enumerate(aug(chunk_audio)):
                        audio_path = f"{monolog_path}/term_{term_index}_aug{aug_index}.wav"
                        audio_paths.append(audio_path)
                        sf.write(audio_path, augm_audio, SAMPLE_RATE)

                    # добавляем описание терма для Whisper
                    monolog_discr["terms"].append({
                        "audio_path": audio_paths[0],  # берем первый аудиофайл (или добавляем все для аугментаций)
                        "start": terms_start,
                        "end": terms_end,
                        "text": "".join(term_words)  # Собираем слова в строку
                    })

                    # начинаем новый терм
                    terms_start = segment_start_time
                    terms_end = segment_end_time
                    term_index += 1

    # добавим файл с описанием монолога в формате для Whisper
    with open(f'{monolog_path}/discriptions.json', 'w') as file:
        json.dump(monolog_discr, file)


In [ ]:
"""
Получаем аудиофайлы длинны по 30 секунд и описание формата:
{
  "speaker": {
    "id": "1",
    "name": ""
  },
  "start": 0.0,
  "end": 33.221,
  "terms": [
    {
      "audio_path": "monologue_0/term_0_aug0.wav",
      "start": 0.0,
      "end": 30.0,
      "text": "...еще большую тревогу вызывает у меня американская политика.."
    },
    {
      "audio_path": "monologue_0/term_1_aug0.wav",
      "start": 30.0,
      "end": 33.221,
      "text": "...в отношении Ближнего Востока.."
    },
    ...
  ]
}

Важно отметить что способ разделения на слова через librosa.effects.split в данном примере не сработал как не сработал бы и pydub.silence.split_on_silence из за того что слова в .ogg аудио записи произносятся потоком почти без пауз между ними. Возможно получилось бы разделить их с помощью алгоритмов распознавания слов Speach to Text.
"""

In [ ]:
"""
Для Whisper нам потребуется файл аннотаций формата
[
    {"audio_filepath": "monologue_1/term_0_aug0.wav", "text": "...еще большую тревогу вызывает у меня американская политика.."},
    {"audio_filepath": "monologue_1/term_1_aug0.wav", "text": "...в отношении Ближнего Востока.."},
    ...
]

Так как поле text у нас уже нормализованно а файлы wav не превосходят 30 секунд по длинне дополнительной обработки не потребуется
"""

In [ ]:
import json

MONOLOGUES_DIR = "monologues"
# аннотации для whisper в формате jsonl
OUTPUT_JSONL = "whisper_dataset.jsonl"

# Сбор всех термов с их аудиофайлами и текстом
entries = []

for monologue_dir in MONOLOGUES_DIR.iterdir():
    desc_path = f"{MONOLOGUES_DIR}/{monologue_dir}/discriptions.json"  # вычисляем пути до всех файлов описаний монологов

    with open(desc_path, "r") as f:
        description = json.load(f)

    for term in description["terms"]:
        entries.append({"audio_filepath": term["audio_path"], "text": term["text"]})

with open(OUTPUT_JSONL, "w") as f:
    for entry in entries:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

# Задание 4

LSTM с input_size = 128 и hidden_size = 256

Сколько для одной LSTM ячейки обучаемых параметров

In [ ]:
"""
LSTM ячейка состоит из 4 гейтов каждый из которых по сути яввляется линейным слоем.

Каждый гейт имеет одну и ту же формулу sigmoid(Wf * Xt + Uf * Ht-1 + Bf)
    здесь Xt размера input_size и Ht-1 размера hidden_size
    следовательно Wf размера hidden_size * input_size, Uf размера hidden_size * hidden_size, Bf размера hidden_size

получаем hidden_size * input_size + hidden_size * hidden_size + hidden_size = hidden_size * (input_size + hidden_size + 1) = 98560
98560 * 4 = 394240 параметров суммаро для 4 гейтов в одной LSTM ячейке
----------------------------------------------------------------------

в torch.nn.LSTM реализованно два биаса для каждого гейта. один для входного слоя другой для скрытого что дает
hidden_size * (input_size + hidden_size + 2) = 98816
98816 * 4 = 395264 параметров суммаро для 4 гейтов в одной torch.nn.LSTM ячейке
-------------------------------------------------------------------------------
"""

In [10]:
import torch.nn as nn

input_size = 128
hidden_size = 256
lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size)  # создаем LSTM блок

In [11]:
# Считаем количество параметров
total_params = sum(p.numel() for p in lstm.parameters() if p.requires_grad)  # сумма параметров
print(f"Обучаемых параметров:", total_params)

Обучаемых параметров: 395264


# Задание 5

Решаем задачу бинарной классификации аудиозаписей

Имеется 100 записей:
* 1 из класса C1
* 99 из класса C0

Обучен классификатор:
* True Positive (Предсказали С1 при истинном С1) с вероятностью 0.9
* False Positive (Предсказали С1 при истинном С0) с вероятностью 0.03

Найти:
1. Какова вероятность что случайную запись из датасета классификатор отнесет к C1
2. Если случайная запись классифицирована как C1 какова вероятность что она действительно C1

In [ ]:
"""
Заметим что вероятность получить класс C1 равна P(C1) = 0.01 а вероятность класса C0 P(C0) = 0.99
Заметим что события С1 истинное и С0 ложное формируют полную группу гипотез

1. По формуле полной вероятности
    P(C1_predicted) = P(C1_predicted | C1_истинная) * P(C1_истинная) + P(C1_predicted | C0_истинная) * P(C0_истинная)
    P(C1_predicted) = 0.9 * 0.01 + 0.03 * 0.99 = 0.0387 ~= 0.038
                                                 --------------

2. По теореме Байеса
    P(C1_истинная | C1_predicted) =   P(C1_predicted | C1_истинная) * P(C1_истинная)                                                   =
                                     _________________________________________________________________________________________________
                                      P(C1_predicted | C1_истинная) * P(C1_истинная) + P(C1_predicted | C0_истинная) * P(C0_истинная)

     = 0.9 * 0.01 / ( 0.9 * 0.01 + 0.03 * 0.99 ) = 0.2325581395 ~= 0.232
                                                   --------------------

Итого:
1. Вероятность того что случайную запись из датасета классификатор отнесет к C1 ~= 0.038
2. Вероятность того что случайная запись классифицирована как C1 и действительно является C1 ~= 0.232
"""

# Задание 6

Для X1 .. Xn из множества вещественных чисел.\
Необходимо написать аналитическое решение для поиска значения b для минимизации sum( (Xi - b)^2 )


In [ ]:
"""
Хотим минимизировать функцию потерь. Для вектора X найти некое b чтобы сумма квадратов разностей стала минимальной
sum( (Xi - b)^2 )

    Для этого воспользуемся градиентом приравненым к нулю т.е. точкой экстремума.
    Т.к функция гладкая и выпуклая (парабола с ветками вверх) следовательно она имеет только одну точку экстремума являющуюся минимумом.
    Т.е

1. Раскрываем скобки под суммой
sum( Xi^2 - 2 * Xi * b + b^2 )

2. Раскроем по линейности суммы
sum(Xi^2) - 2 * b * sum(Xi) + N * b^2

3. найдем частную производную по b
-2 * sum(Xi) + N * 2b

4. Приравняем производную к нулю для поиска точки экстремума
-2 * sum(Xi) + N * 2b = 0
N * 2b = 2 * sum(Xi)
2 * b = 2 * 1/N * sum(Xi)
b = 1/N * sum(Xi)

Ответ: (оказалось что выгоднее всего с точки зрения минимизации функции потерь выбирать среднее значение для b)
b = 1/N * sum(Xi)
---------------------------------------------------------------------------------------------------------------
"""

# Задание 7

Образуют ли векторы:
* x1 = [1, 0, 1]^T
* x2 = [1, 1, 0]^T
* x3 = [1, 1, 1]^T

Базис векторного пространства в R^3. Написать аналитическое решение

In [ ]:
"""
Проверим векторы на линейную зависимость.
Если окажется что один из векторов представим как линейная комбинация двух других то вектора линейно зависимы -> не образуют базис в R^3

1. Перепишем вектры как линейную комбинацию и приравняем к нулю:
a * [1, 0, 1]^T + b * [1, 1, 0]^T + c * [1, 1, 1]^T = 0

    тогда получится что любой вектор представим как линейная комбинация двух других со знаком минус например:
    a * [1, 0, 1]^T + b * [1, 1, 0]^T = -c * [1, 1, 1]^T

2. Составим и решим систему уравнений
    [1]       [1]       [1]   [0]
a * [0] + b * [1] + c * [1] = [0]
    [1]       [0]       [1]   [0]

 /  a + b + c = 0
<   b + c = 0
 \  a + c = 0

    из второй и третей строки получим:
    b = -c
    a = -c
    подставим в первое:
    -с -с + с= 0
    т.е. -с = 0
    Тогда единственное решение системы будет в точке [0, 0, 0]^T (a=b=c=0)

    следовательно Векторы можно представить как линейную комбинацию друг друга только если все они нулевые.
                  Значит они линейно независимы

А это значит что векторы образуют базис векторного пространства в R^3
----------------------------------------------------------------------
"""